In [70]:
import pandas as pd
import panel as pn 
import hvplot.pandas 
import holoviews as hv 
from holoviews import opts
import matplotlib.pyplot as plt

pn.extension('tabulator')

# Load data and cache
if 'data' not in pn.state.cache.keys():
    df = pd.read_csv('https://raw.githubusercontent.com/SumanGouda/Data-Visualization-Using-Python/refs/heads/main/PANEL/NETFLIX%20DATA%20ANALYSIS%20PROJECT/DATA/netflix_titles.csv')
    pn.state.cache['data'] = df.copy()
else:
    df = pn.state.cache['data']
   

In [86]:
df.columns
df['year'] = pd.to_datetime(df['date_added'],errors='coerce').dt.year.astype('Int64')
df['year'].unique() 

<IntegerArray>
[2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2009, 2008,
 <NA>, 2010]
Length: 15, dtype: Int64

In [72]:
# Define widgets
year_slider = pn.widgets.IntSlider(
    name='Year', start=1925, end=2021,
    value=2020, step=1, bar_color='#FF6347'
)

## Number of released based on the Genre 

In [13]:
rel_type = pn.widgets.RadioButtonGroup(
    name='Type',
    options=['TV Show', 'Movie'],
    value='TV Show',
    button_type='success'
)

# Create function for top genres horizontal bar chart
@pn.depends(year_slider, rel_type)
def genre_bar(year, typ):
    # Filter data by year and type
    subset = df[(df['release_year'] == year) & (df['type'] == typ)].copy()

    if subset.empty:
        return pn.pane.Markdown(f"### ⚠️ No data for {typ}s in {year}.")

    # Split and count genres
    genres = subset['listed_in'].dropna().str.split(', ')
    exploded = genres.explode()
    genre_counts = exploded.value_counts().head(10).reset_index()
    genre_counts.columns = ['genre', 'count']

    # Create horizontal bar chart
    plot = genre_counts.hvplot.barh(
        x='genre',
        y='count',
        title=f'Top 10 Genres ({typ}s in {year})',
        height=500,
        width=600,
        color='crimson',
        ylabel='Genre',
        xlabel='Number of Titles',
        ysort=True,
        legend=None
    )

    # Apply styling with tick color hook
    return plot.opts(
        fontsize={
            'ticks': '11pt',
            'labels': '13pt',
            'title': '15pt'
        },
        hooks=[
            lambda p, _: (
                setattr(p.handles['xaxis'], 'major_label_text_color', '#FF6347'),
                setattr(p.handles['yaxis'], 'major_label_text_color', '#FF6347'),
                setattr(p.handles['xaxis'], 'axis_label_text_color', "#1F0602"),
                setattr(p.handles['yaxis'], 'axis_label_text_color', "#1F0602"),
                setattr(p.handles['plot'].title, 'text_color', "#3C1006")
            )
        ]
    )

## Total Number Of Release 


In [ ]:
@pn.depends(year_slider)
def plot_pie(year):
    num_movies = df[(df['release_year'] == year) & (df['type'] == 'Movie')].shape[0]
    num_shows = df[(df['release_year'] == year) & (df['type'] == 'TV Show')].shape[0]

    fig, ax = plt.subplots()
    ax.pie(
        [num_movies, num_shows],
        labels=['Movies', 'TV Shows'],
        textprops={'fontsize': 14, 'color': "#170503"},
        autopct='%1.1f%%',
        startangle=45,
        pctdistance=0.8,
        colors=['#FF6F61', '#6A5ACD'],
        wedgeprops={'width': 0.4}
    )

    ax.set_title(f"Content Distribution in {year}", fontsize=14, color='#170503', weight='bold')
    return fig

fig = plot_pie(2020)
plt.show()

In [ ]:

df['country'] = df['country'].fillna('Unknown')
country_options = ['All'] + sorted(df['country'].unique().tolist())
type_options = ['All', 'Movie', 'TV Show']

# 📦 Widgets
show_type = pn.widgets.RadioButtonGroup(
    name='Type',
    options=type_options,
    button_type='success'
)

country = pn.widgets.Select(
    name='Country',
    options=country_options
)

@pn.depends(country, show_type)
def plot_graph (select_country, select_show):
    data = df.copy()
    if (select_country == 'All') and (select_show == 'All'):
        grouped_movies = data[data['type'] == 'Movie'].groupby('year').size()
        grouped_show = data[data['type'] == 'TV Show'].groupby('year').size()
        pivot_df = pd.concat([grouped_movies, grouped_show], axis=1)
        pivot_df.columns = ['Movies', 'TV Shows']
        pivot_df = pivot_df.fillna(0).sort_index()
        
        # Plot area chart
        plot = pivot_df.hvplot.area(
            x='year',
            y=['Movies', 'TV Shows'],
            stacked=True,
            alpha=0.7,
            color=['#E53935', '#1E88E5'],
            ylabel='Number of Titles',
            title='Netflix Content Release Trends (Movies & TV Shows)'
        )
        return plot
    
    elif (select_country != 'All') and (select_show == 'All'):
        data = data[data['country'] == select_country]
        grouped_movies = data[data['type'] == 'Movie'].groupby('year').size()
        grouped_show = data[data['type'] == 'TV Show'].groupby('year').size()
        pivot_df = pd.concat([grouped_movies, grouped_show], axis=1)
        pivot_df.columns = ['Movies', 'TV Shows']
        pivot_df = pivot_df.fillna(0).sort_index()
        
        # Plot area chart
        plot = pivot_df.hvplot.area(
            x='year',
            y=['Movies', 'TV Shows'],
            stacked=True,
            alpha=0.7,
            color=['#E53935', '#1E88E5'],
            ylabel='Number of Titles',
            title='Netflix Content Release Trends (Movies & TV Shows)'
        )
        return plot
    elif (select_country != 'All') and (select_show != 'All'):
        data = data[(data['country'] == select_country) & (data['type'] == select_show)]
        value = data.groupby('year').size().reset_index(name='count')
        
        # Plot area chart
        plot = value.hvplot.area(
            x='year',
            y='count',
            stacked=True,
            alpha=0.7,
            color=['#E53935', '#1E88E5'],
            ylabel='Number of Titles',
            title='Netflix Content Release Trends (Movies & TV Shows)'
        )
        return plot
    elif (select_country == 'All' ) and (select_show != 'All'):
        data = data[data['type'] == select_show]
        value = data.groupby('year').size().reset_index(name='count')
        # Plot area chart
        plot = value.hvplot.area(
            x='year',
            y='count',
            stacked=True,
            alpha=0.7,
            color=['#E53935', '#1E88E5'],
            ylabel='Number of Titles',
            title='Netflix Content Release Trends (Movies & TV Shows)'
        )
        return plot

:Area   [year]   (count)

In [ ]:
import hvplot.pandas

# Group and reshape data
grouped = df.groupby(['year', 'type']).size().reset_index(name='count')

# Pivot so each type becomes a column
pivot_df = grouped.pivot(index='year', columns='type', values='count').fillna(0)

# Plot stacked bar chart
pivot_df.hvplot.bar(
    stacked=True,
    height=500,
    width=700,
    xlabel='Year',
    ylabel='Number of Titles',
    title='Netflix Releases by Type Over Years',
    color=['#FF6F61', '#6A5ACD']
)

In [ ]:
pn.extension(raw_css=[
    """
    body {
        background-color: #111;
        color: #fff;
        font-family: 'Segoe UI', sans-serif;
    }

    .bk-root {
        border-radius: 10px;
        box-shadow: 0 0 12px rgba(255, 255, 255, 0.15);
    }

    button {
        background-color: #ff0044;
        color: white;
        font-weight: bold;
        border-radius: 6px;
    }
    """
]) 
layout = pn.Column(
    pn.Row(year_slider, rel_type),
    pn.Row(plot_graph, genre_bar),
    normalized_plot
    
)
layout.servable()